# Working with Data via MXNet NDArray

MXNet's NDArray is a tensor data structure similar to numpy's multi-dimensional array. 
It confers a few key advantages. First, it supports asynchronous computation on CPU and GPU, and distributed cloud architectures. Second, it provides support for automatic differentiation. These properties make it especially suited as a library for both deep learning research and production.


## Getting Started

First, let's import mxnet and (for convenience) mxnet.ndarray, the only dependencies we'll need in this tutorial.

In [92]:
import mxnet 
import mxnet.ndarray as nd

First, we'll create a 6x4 matrix, without initializing values

In [110]:
x = nd.empty(shape=(6,4))
print(x)

[[  0.00000000e+00  -2.00000000e+00  -5.40732375e+05   1.58494897e+29]
 [  8.40779079e-45   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.87544948e+28   4.15857830e-05]
 [  2.67017140e-06   1.28024709e-11   5.26692006e-08   2.14771091e-04]
 [  1.34215661e-05   4.03291693e-08   1.06061622e-08   5.39295324e-05]
 [  4.12020557e-11   7.14495034e+31   4.14181787e-41   5.51012977e-40]]
<NDArray 6x4 @cpu(0)>


We can also create a 6x5 matrix, initialized with values drawn from a standard normal distribution.

In [111]:
x = nd.random_normal(shape=(6,4))
print(x)

[[-0.75670922  0.34593368  0.13926521 -1.78783298]
 [-0.20225924 -0.17870884  1.04662514  0.98684961]
 [-2.31759715 -0.82165414  0.22842792  1.18865573]
 [-0.55536991 -0.51751894 -0.22436801 -0.21166277]
 [-0.03407707 -1.07040215 -0.86891282  1.33646619]
 [ 0.83015364  0.77279937  0.22461908 -0.36800992]]
<NDArray 6x4 @cpu(0)>


As in NumPy, we can get its shape

In [119]:
print(x.shape)

(6, 4)


We can also query it's size, which is equal to the product of the shape.

In [120]:
print(x.size)

24


## Operations

There are multiple syntaxes for operations. Let’s see addition as an example

In [96]:
y = nd.random_normal(shape=(5,3))
c = x + y
print(c)

[[ 0.14507043 -3.30073404 -0.2066054 ]
 [-0.22305712 -1.51432788  2.18144083]
 [-0.66082555  1.42019582  1.78058803]
 [-1.6503545  -1.23633528 -0.18388587]
 [ 0.71802235  1.87202454 -0.51498145]]
<NDArray 5x3 @cpu(0)>


In the previous example, MXNet allocates new memory for the sum ``x+y`` and assigns a reference to the variable ``c``. To make better use of memory, we often prefer to perform operations in place, reusing already allocated memory. 

In MXNet, we can perform inplace operations by assigning the results of operations using slice notation (``result[:] = ...``).

In [97]:
result = nd.zeros(shape=(5,3))
result[:] = x+y
print(result)

[[ 0.14507043 -3.30073404 -0.2066054 ]
 [-0.22305712 -1.51432788  2.18144083]
 [-0.66082555  1.42019582  1.78058803]
 [-1.6503545  -1.23633528 -0.18388587]
 [ 0.71802235  1.87202454 -0.51498145]]
<NDArray 5x3 @cpu(0)>


## Slicing

MXNet NDArrays currently support slicing along the first (0th) axis only. To use arbitrary slicing patterns, you can convert NDArrays to numpy arrays and back.

In [98]:
result[2:4]

[[-0.66082555  1.42019582  1.78058803]
 [-1.6503545  -1.23633528 -0.18388587]]
<NDArray 2x3 @cpu(0)>

## Converting from MXNet NDArray to NumPy Array

Converting MXNet NDArrays to and from NumPy is easy. Note that the converted arrays do not share memory.

In [99]:
a = nd.ones(shape=(5))
print(a)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [100]:
b = a.asnumpy()
print(b)

[ 1.  1.  1.  1.  1.]


In [101]:
z[0] = 2
print(z)
print(a)

[ 2.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Converting from NumPy Array to MXNet NDArray

Constructing an MXNet NDarray from a NumPy Array is straightforward.

In [102]:
c = nd.array(b)
print(c)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


## Managing Context

In MXNet, every array has a context. A context could be the CPU, or one of many GPUs. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. 


In [47]:
d = nd.array(b, mxnet.cpu())

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [103]:
e = d.copyto(mx.cpu(1))
print(e)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(1)>


Often we only want to make a copy if the variable isn't already on the desired context. In these cases, we can call ``as_in_context()`` which only makes a copy when the source and target contexts are different.

In [106]:
f = d.as_in_context(mxnet.cpu(0))
print(f)

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>
